In [158]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler  # 导入标准化模块

In [159]:
# 定义神经网络模型
class Net(nn.Module):
    def __init__(self, input_size, output_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)  # 第一个隐藏层
        self.fc2 = nn.Linear(64, 32)           # 第二个隐藏层
        self.fc3 = nn.Linear(32, output_size)  # 输出层

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [160]:
data = pd.read_excel("京津冀3省2014-2023使用版数据-2位小数.xlsx")

x, y = data.iloc[:,2:].to_numpy(), data.iloc[:,1].to_numpy().reshape(-1, 1)
scaler = MinMaxScaler()  # 实例化
X = scaler.fit_transform(x)  # 标准化特征
Y = scaler.fit_transform(y)  # 标准化标签

train_X, train_Y, val_X, val_Y = X[:-2], Y[:-2], X[-2:], Y[-2:]
train_torch_data = TensorDataset(torch.FloatTensor(train_X), torch.FloatTensor(train_Y))
val_torch_data = TensorDataset(torch.FloatTensor(val_X), torch.FloatTensor(val_Y))
all_torch_data = TensorDataset(torch.FloatTensor(X), torch.FloatTensor(Y))

train_loader = DataLoader(train_torch_data, batch_size=4, shuffle=True)
val_loader = DataLoader(val_torch_data, batch_size=1)
all_loader = DataLoader(all_torch_data, batch_size=data.shape[0])




In [161]:
# 将数据分为输入和输出
input_size = 2  # 5个输入
output_size = 1  # 1个输出
learning_rate = 1e-5 # 学习率

bpnn = Net(input_size, output_size)

optimizer = optim.Adam(bpnn.parameters(), learning_rate)  # 使用Adam算法更新参数
criteon = torch.nn.MSELoss()  # 损失函数，回归问题采用均方误差

In [162]:

# 训练模型
n_epochs = 10000
for epoch in range(n_epochs):
    for X_batch, Y_batch in train_loader:
        optimizer.zero_grad()
        y_pred = bpnn(X_batch)
        loss = criteon(y_pred, Y_batch)
        loss.backward()
        optimizer.step()

    if (epoch+1)%100 == 0:
        print(f"Epoch {epoch+1} , Loss: {loss.item()}")

Epoch 100 , Loss: 0.03243058919906616
Epoch 200 , Loss: 0.016658086329698563
Epoch 300 , Loss: 0.19837889075279236
Epoch 400 , Loss: 0.17671440541744232
Epoch 500 , Loss: 0.02013644017279148
Epoch 600 , Loss: 0.141000434756279
Epoch 700 , Loss: 0.13206005096435547
Epoch 800 , Loss: 0.12293755263090134
Epoch 900 , Loss: 0.02392258495092392
Epoch 1000 , Loss: 0.025491736829280853
Epoch 1100 , Loss: 0.026092763990163803
Epoch 1200 , Loss: 0.020284997299313545
Epoch 1300 , Loss: 0.12252125144004822
Epoch 1400 , Loss: 0.022963162511587143
Epoch 1500 , Loss: 0.09983505308628082
Epoch 1600 , Loss: 0.1121002584695816
Epoch 1700 , Loss: 0.11707644164562225
Epoch 1800 , Loss: 0.03194352611899376
Epoch 1900 , Loss: 0.013202950358390808
Epoch 2000 , Loss: 0.09555337578058243
Epoch 2100 , Loss: 0.028223160654306412
Epoch 2200 , Loss: 0.10260731726884842
Epoch 2300 , Loss: 0.10711566358804703
Epoch 2400 , Loss: 0.09570245444774628
Epoch 2500 , Loss: 0.02750147320330143
Epoch 2600 , Loss: 0.024263925

In [163]:
# 输出验证集结果
for X_val, y_val in val_loader:
    val_y_pred = bpnn(X_val)
    loss = criteon(val_y_pred, y_val)
    print(f"预测值： {scaler.inverse_transform(val_y_pred.detach().numpy())[0][0]} ， "
          f"MSE: {loss.item()}")

预测值： 2.657160520553589 ， MSE: 0.016120092943310738
预测值： 2.9305975437164307 ， MSE: 0.1196301132440567


In [164]:
# 计算整体MAPE
for X, y in all_loader:
    y_pred = bpnn(X)
    loss = criteon(y_pred, y)
    print(f"MSE: {loss.item()}")
    result = scaler.inverse_transform(y_pred.detach().numpy())

MSE: 0.03177163004875183


In [165]:
# 整体MAPE
APE = np.abs(result.reshape(-1) - data.iloc[:, 1].to_numpy()) / data.iloc[:, 1].to_numpy()
MAPE = np.mean(APE)
print(APE)
print("MAPE: ", MAPE)

[0.1347677  0.02298366 0.11201859 0.07384426 0.06425892 0.00317876
 0.05364102 0.10934165 0.05321909 0.14260789]
MAPE:  0.07698615294783473


In [166]:
# 后两项MAPE
print(np.mean(APE[-2:]))
pd.DataFrame(result).to_excel('BPNN_hatx0-case2.xlsx')

pd.DataFrame(APE).to_excel('BPNN_APE-case2.xlsx')

0.09791349027732163
